In [ ]:
import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

import pandas as pd

from hist import Hist


In [ ]:
start = time.time()


# At Siena
#filename = '/mnt/qnap/babar_data/test.parquet'
topdir = '/mnt/qnap/babar_data/bnv_plambda'
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")


In [ ]:
h = Hist.new.Reg(100, 5.2, 5.3, name="BpostFitMes", label=r"M$_{ES}$ [GeV/c$^2$]") \
         .StrCat([], name="SP", label="SP modes", growth=True)\
         .StrCat([], name="cuts", label="Cuts", growth=True)\
         .Weight()


bkg_spmodes = ['998','1005', '1235', '1237', '991']
spmodes = bkg_spmodes + ['-999']
#spmodes = ['991']

for spmode in spmodes:
    mask = data.spmode == spmode
    x = data[mask]['BpostFitMes'][:,0]

    weight = 1
    if spmode == '-999':
        weight = 0.02
    
    h.fill(BpostFitMes=x, SP=spmode, cuts=f"0", weight=weight)

In [ ]:
h[:,:,:].stack('SP')[:].project('BpostFitMes').plot(stack=True, histtype="fill")
h[:,['-999'],:].project('BpostFitMes').plot(histtype="errorbar", color='black')

plt.legend()

In [ ]:
h[:,bkg_spmodes,:].stack('SP')[:].project('BpostFitMes').plot(stack=True, histtype="fill")
h[:,['-999'],:].project('BpostFitMes').plot(histtype="errorbar", color='black')

plt.legend()

# Automate histogram creation

*Not histogram creation*

In [ ]:
#data.fields

In [ ]:
hist_defs = {}
hist_defs['BSphr'] =         {"nbins":100, "lo":0, "hi":0.2, "label":"Sphericity"}
hist_defs["BpostFitDeltaE"]= {"nbins":100, "lo":-1.0, "hi":1.0, "label":"$\Delta$ E"}
hist_defs["BpostFitMes"]=    {"nbins":100, "lo":5.0, "hi":5.3, "label":"$M_{ES}$ [GeV/c$^2$]"}
hist_defs["BpostFitMes"]=    {"nbins":100, "lo":5.0, "hi":5.3, "label":"$M_{ES}$ [GeV/c$^2$]"}

hist_defs

In [ ]:
def create_empty_histograms(hist_defs): 
    all_hists= {}
    
    for var in hist_defs.keys():
        h = Hist.new.Reg(hist_defs[var]["nbins"], hist_defs[var]["lo"], hist_defs[var]["hi"], name='var', label=f"{hist_defs[var]['label']}") \
                 .StrCat([], name="SP", label="SP modes", growth=True)\
                 .StrCat([], name="cuts", label="Cuts", growth=True)\
                 .Weight()
    
        all_hists[var] = h

    return all_hists


def fill_histograms(data, empty_hists, spmodes=['998']):
    for key in empty_hists.keys(): 
        print(key)

        for spmode in spmodes:
            weight = 1
            if spmode=='-999':
                weight = 0.05
                
            sp_mask= data.spmode== spmode
            x= ak.flatten(data[sp_mask][key])
            empty_hists[key].fill(var=x, SP= spmode, cuts= f"0", weight= weight)

    return 1


In [ ]:
all_hists = create_empty_histograms(hist_defs)

all_hists

In [ ]:
fill_histograms(data, all_hists, spmodes=['998', '1005', '1235', '1237', '-999'])

In [ ]:
all_hists

In [ ]:
h = all_hists['BSphr']

h[:,['998'],:].project('var').plot(histtype="errorbar", color='black')

In [ ]:
x = 10

print(x)

x = 20

print(x)

In [ ]:
x = 10

print(x)

y = x

print(y)

x = 20

print(x,y)

In [ ]:
x = [10]

print(x)

y = x

print(y)

x[0] = 20

print(x,y)

In [ ]:
x = np.array([10])

print(x)

y = x

print(y)

x[0] = 20

print(x,y)



y[0] = 999

print(x,y)

In [ ]:
# Make a copy of the actual values and give it a new name!
x = [10]

print(x)

y = list(x)

print(y)

x[0] = 20

print(x,y)

In [ ]:
# Make a new array!
x = np.array([10])

print(x)

y = x.copy()

print(y)

x[0] = 20

print(x,y)

# Automate plotting

In [ ]:
#data.fields

In [ ]:
h = Hist.new.Reg(100, 0, .2, name="BSphr", label=r"Sphericity") \
         .StrCat([], name="SP", label="SP modes", growth=True)\
         .StrCat([], name="cuts", label="Cuts", growth=True)\
         .Weight()

#for key,value in data.items():
#    x = value['BpostFitMes'][:,0]
#    h.fill(BpostFitMes=x, SP=key, cuts=f"0", weight=1)

spmodes = ['998','1005', '1235', '1237', '3981']
for spmode in spmodes:
    mask = data.spmode == spmode
    x = data[mask]['BSphr'][:,0]
    h.fill(BSphr=x, SP=spmode, cuts=f"0", weight=1)

In [ ]:
h.stack('SP')[:].project('BSphr').plot(stack=True, histtype="fill")

plt.legend()

In [ ]:
plt.figure(figsize=(18,12))

spmodes = ['998','1005', '1235', '1237', '3981']
for idx,spmode in enumerate(spmodes):
    plt.subplot(2,3,idx+1)
    h[:,spmode,:].project('BSphr').plot(histtype="fill", label=spmode)
    plt.legend()

In [ ]:
def plot_histograms(all_hists, vars=[], spmodes=['998'], save= True):

    if len(vars) == 0:
        vars = list(all_hists.keys())
    cd= {}
    cd["998"]=  {"tab:blue"}
    cd["1005"]= {"tab:orange"}
    cd["-999"]= {"tab:green"}
    cd["1235"]= {"tab:red"}
    cd["1237"]= {"tab:purple"}
    cd["3981"]= {"tab:pink"}


    for var in vars:
        
        h = all_hists[var]
        #print(h)
        #h[:,['998'],:].project('var').plot(histtype="errorbar", color='black')
        plt.figure(figsize=(18,12))
        
        plt.subplot(2,3,1)

        h[:,:,:].stack('SP')[:].project('var').plot(stack=True, histtype="fill")
        #h[:,spmodes,:].project('var').plot(histtype="errorbar", color='black')
        
        plt.legend()
        plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
        
        for idx,spmode in enumerate(spmodes):
            plt.subplot(2,3,idx+2)
            h[:,spmode,:].project('var').plot(histtype="fill", label=spmode, color= cd[str(spmode)])
            plt.legend()
            plt.xlabel(plt.gca().get_xlabel(), fontsize=18)
    
            if save== True:
                # Let's save them in a directory called `plots`
                # Look up how to first, check (using python) if a directory exists
                # and if it does not, then create that directory (from python)
                plt.savefig(f"plot_hist_{var}.png")
    

In [ ]:
all_hists

In [ ]:
plot_histograms(all_hists, spmodes=['998', '1005', '1235', '1237', '-999']);

# Blinding and fitting region

In [ ]:
#data['spmode']

In [ ]:
mask = data['Lambda0FlightLen']>=0

sigmask = data['spmode'] == '-999'


mes = ak.flatten(data['BpostFitMes'][mask & sigmask])
de  = ak.flatten(data['BpostFitDeltaE'][mask & sigmask])


plt.figure(figsize=(12,12))
plt.subplot(3,2,1)
plt.plot(mes, de, '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

norg = len(mes)
print(f"# original: {norg}")

# Stuff
blinding_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 
s
plt.subplot(3,2,3)
plt.plot(mes[blinding_mask], de[blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,4)
plt.plot(mes[~blinding_mask], de[~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


plt.subplot(3,2,5)
plt.plot(mes[fitarea_mask], de[fitarea_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,6)
plt.plot(mes[fitarea_mask & ~blinding_mask], de[fitarea_mask & ~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


In [ ]:
mask = data['Lambda0FlightLen']>=0

sigmask = data['spmode'] != '-999'


mes = ak.flatten(data['BpostFitMes'][mask & sigmask])
de  = ak.flatten(data['BpostFitDeltaE'][mask & sigmask])


plt.figure(figsize=(12,12))
plt.subplot(3,2,1)
plt.plot(mes, de, '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

norg = len(mes)
print(f"# original: {norg}")

# Stuff
blinding_mask = (mes>5.27) & ((de>-.07) & (de<.07)) 
fitarea_mask   =  (mes>5.2) & ((de>-.2) & (de<.2)) 

plt.subplot(3,2,3)
plt.plot(mes[blinding_mask], de[blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,4)
plt.plot(mes[~blinding_mask], de[~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")


plt.subplot(3,2,5)
plt.plot(mes[fitarea_mask], de[fitarea_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

plt.subplot(3,2,6)
plt.plot(mes[fitarea_mask & ~blinding_mask], de[fitarea_mask & ~blinding_mask], '.', markersize=0.5, alpha=0.5)
plt.xlim(5.2, 5.3)
plt.ylim(-0.5, 0.5)

nblind = len(mes[blinding_mask])
print(f"# blinded: {nblind}")

nnotblind = len(mes[~blinding_mask])
print(f"# notblinded: {nnotblind}")

print(f"frac blinded: {nblind/norg}")
